In [ ]:
%pip install -U sentence-transformers

In [1]:
import os, sys
import argparse
import ast

from collections import Counter
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer, AutoModelForTokenClassification

import advertools as adv
from bertopic import BERTopic

c:\Users\Chloe\Anaconda3\envs\summ\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('sectioned_data_2022.csv')
data

,Unnamed: 0,ecli,date,inhoudsindicatie,procesverloop,overwegingen,beslissing,sections
0,0,ECLI:NL:CBB:2022:1,2022-01-11,"\n\nArtikel 2:3, eerste lid, van de Algemene w...",Procesverloop Bij besluit van 2 oktober 2020 (...,Overwegingen 1.1 Appellant heeft op 10 mei 202...,Beslissing Het College: - verklaart het beroep...,"['Overwegingen', '1.1 Appellant heeft op 10 me..."
1,1,ECLI:NL:CBB:2022:10,2022-01-18,\n\nMond - en klauwzeer. Medio maart 2001 is i...,Procesverloop Bij besluit van 29 maart 2001 (h...,Overwegingen Feiten en omstandigheden 1. Het C...,Beslissing Het College: verklaart het beroep g...,"['Overwegingen Feiten en omstandigheden', ' 1...."
2,2,ECLI:NL:CBB:2022:100,2022-03-08,\n\n-\tBeleidsregel tegemoetkoming ondernemers...,Procesverloop Bij besluit van 3 februari 2021 ...,Overwegingen 1. Het wettelijk kader is opgenom...,Beslissing Het College: - verklaart het beroep...,"['Overwegingen', ' 1. Het wettelijk kader is o..."
3,3,ECLI:NL:CBB:2022:101,2022-03-08,"\nWarenwet, hoger beroep, Verordening 853/2004...",Procesverloop in hoger beroep De minister heef...,Grondslag van het geschil 1.1 De rechtbank hee...,Beslissing Het College bevestigt de aangevalle...,"['Grondslag van het geschil', '1.1 De rechtban..."
4,4,ECLI:NL:CBB:2022:102,2022-03-08,\nRegeling garanties van oorsprong en certific...,Procesverloop Bij afzonderlijke besluiten van ...,Overwegingen Inleiding 1.1 Appellanten exploit...,Beslissing Het College: verklaart de beroepen ...,"['Overwegingen Inleiding', '1.1 Appellanten ex..."
...,...,...,...,...,...,...,...,...
95,95,ECLI:NL:CBB:2022:206,2022-05-03,\nISDE-subsidie voor warmtepompen. Aanvragen a...,Procesverloop Bij 32 afzonderlijke besluiten v...,Overwegingen In de periode van 12 maart 2019 t...,Beslissing Het College verklaart het beroep on...,['Overwegingen In de periode van 12 maart 2019...
96,96,ECLI:NL:CBB:2022:207,2022-05-03,\nSDE-subsidie voor co-vergistingsinstallaties...,Procesverloop Appellanten hebben beroep ingest...,Overwegingen 1.1 Verweerder heeft aan appellan...,Beslissing Het College verklaart de beroepen o...,"['Overwegingen', '1.1 Verweerder heeft aan app..."
97,97,ECLI:NL:CBB:2022:208,2022-05-03,\nVerzoek om schadevergoeding wordt afgewezen....,Procesverloop Verzoeker heeft op 4 december 20...,Overwegingen Relevante bepalingen 1.\tOp grond...,Beslissing Het College wijst het verzoek tot s...,"['Overwegingen Relevante bepalingen', ' 1.\tOp..."
98,98,ECLI:NL:CBB:2022:209,2022-05-03,\n\nFosfaatrechten – verzoek schadevergoeding\...,Procesverloop Op 29 oktober 2020 heeft verzoek...,Overwegingen Relevante bepalingen 1.1 Op grond...,Beslissing Het College: veroordeelt verweerder...,"['Overwegingen Relevante bepalingen', '1.1 Op ..."


In [3]:
docs = [word for row in data['sections'] for word in ast.literal_eval(row)]
print(len(docs))
docs = [s for s in docs if len(s.split()) >= 10]
print(len(docs))
#docs = data.fulltext.tolist()
print(docs[90:130])

2633
2248
['93.11.9. Het College beantwoordt die vraag ontkennend. Verweerder heeft in bezwaar gekeken of de tweede SBI-code vermeld in het handelsregister (SBI-code', '93.11.9) de hoofdactiviteit is. Verweerder vindt de hoofdactiviteit van appellante beter passen bij de SBI-code', '68.20.4, omdat appellante omzet genereert uit de verhuur van de klimhallen. Daarom heeft verweerder op basis van de SBI-code', '68.20.4 bij het bestreden besluit een subsidie toegekend. Ter zitting heeft appellante echter afdoende toegelicht dat zij niet alleen de klimwanden verhuurt, maar ook exploiteert. Appellante bepaalt de uitstraling van de klimhallen en heeft het merk gecreëerd. Daarnaast heeft appellante vergaande invloed op de dagelijkse gang van zaken bij de verschillende klimhallen, zoals het verzorgen van de opleiding van personeel en de veiligheidsaspecten, en waar nodig werken de eigenaren van appellante zelf in de accommodaties. Hieruit kan worden afgeleid dat zowel de ondernemingen die tot d

In [4]:
stopwords_dutch = list(adv.stopwords['dutch'])

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("GroNLP/gpt2-small-dutch")

2024-04-10 10:14:20,017 | INFO | SentenceTransformer.py:107 | __init__ | Load pretrained SentenceTransformer: GroNLP/gpt2-small-dutch
2024-04-10 10:14:20,613 | WARNING | SentenceTransformer.py:1129 | _load_auto_model | No sentence-transformers model found with name GroNLP/gpt2-small-dutch. Creating a new one with MEAN pooling.
2024-04-10 10:14:21,529 | INFO | SentenceTransformer.py:213 | __init__ | Use pytorch device_name: cuda


In [6]:
model.tokenizer.pad_token = model.tokenizer.eos_token
sentence_embeddings = model.encode(docs, show_progress_bar=True)

Batches: 100%|██████████| 71/71 [01:29<00:00,  1.26s/it] 


In [10]:
from umap import UMAP
from hdbscan import HDBSCAN

n_neighbors = 5
n_components = 10
min_dist = 0.01
metric = 'cosine'
min_cluster_size = 5
cluster_selection_method = 'eom'

umap_model = UMAP(n_neighbors = n_neighbors, n_components=n_components, min_dist=min_dist, metric=metric, random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean',cluster_selection_method=cluster_selection_method, prediction_data=True, cluster_selection_epsilon=0.15)
vectorizer_model = CountVectorizer(stop_words=stopwords_dutch, min_df=2, ngram_range=(1, 2))


topic_model = BERTopic(
    language='multilingual',
    calculate_probabilities=True,

    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    top_n_words=10,
    verbose=True,
    nr_topics=15,
)
topics, probs = topic_model.fit_transform(docs, sentence_embeddings)
print(topics)

2024-04-10 10:16:55,988 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


2024-04-10 10:17:03,060 - BERTopic - Dimensionality - Completed ✓
2024-04-10 10:17:03,062 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-10 10:17:04,684 - BERTopic - Cluster - Completed ✓
2024-04-10 10:17:04,685 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-10 10:17:05,289 - BERTopic - Representation - Completed ✓
2024-04-10 10:17:05,290 - BERTopic - Topic reduction - Reducing number of topics
2024-04-10 10:17:05,857 - BERTopic - Topic reduction - Reduced number of topics from 132 to 15


[-1, 0, 4, 0, 0, 0, -1, -1, 0, 4, 0, -1, 0, 0, -1, -1, 0, -1, 1, 0, 0, -1, -1, 1, -1, 1, -1, -1, 0, 0, 0, 0, -1, -1, -1, -1, 3, 0, 3, 3, 1, -1, -1, 0, 0, 0, 1, 1, -1, 0, -1, 0, 1, 2, -1, 0, 1, 1, 0, 1, 1, 0, 0, 5, 1, -1, 0, 4, -1, 0, 0, 0, 4, 1, 2, 0, 1, 0, 5, -1, 0, -1, 0, 2, -1, 2, 0, 0, -1, -1, 2, -1, 5, 0, 0, 7, -1, 0, 2, 0, 0, 0, 0, 0, 0, 1, -1, 1, 1, 1, 0, 0, 1, 11, 1, 8, 8, 0, 8, 0, 8, 8, 8, 8, 3, 3, 0, 8, 0, 0, 0, 0, 2, 0, 0, -1, 7, 3, 1, -1, 3, 0, -1, 0, 0, -1, 1, -1, 1, 1, 1, 6, 0, 3, 2, 7, 0, 0, 0, 2, 2, 0, 4, 0, 1, 0, 1, -1, -1, 1, 1, 3, 1, 1, 1, -1, 1, -1, 0, 0, 1, 1, 3, 2, 2, 0, 2, 4, 0, 4, 0, 0, -1, 0, 0, 0, 9, -1, 6, 0, 1, 0, 0, 1, 12, 2, 2, 1, 0, 1, 1, -1, 1, 0, 1, 0, -1, 0, 3, 0, 1, -1, 4, 4, 2, 5, 1, 1, 5, 3, 0, 1, 0, -1, 0, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, -1, 3, 0, -1, 3, -1, 0, 1, 1, 0, -1, 0, 1, -1, 0, 1, -1, -1, -1, -1, 0, -1, 1, 1, 0, -1, -1, 6, 1, 1, 1, 0, 1, 1, 1, 1, -1, 1, 0, -1, -1, 0, 0, 0, -1, -1, 1, 7, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, 1, 0, 0, 1, 1

In [11]:
topic_model.topic_labels_

{-1: '-1_appellante_appellant_college_verweerder',
 0: '0_naam_verweerder_appellante_college',
 1: '1_college_lid_acm_kosten',
 2: '2_naam_verweerder_mailadres_college',
 3: '3_lid_eerste_eerste lid_ten',
 4: '4_bestaat aanleiding_bestaat_aanleiding_ongegrond proceskostenveroordeling',
 5: '5_rot 18_rot_naam_18',
 6: '6_lid besluit_lid_dieren_houders dieren',
 7: '7_belang zijnde_uitgebreide_verloop procedure_verloop',
 8: '8_dieren_biomassa_zuivere biomassa_zuivere',
 9: '9_aansluiting schadevergoedingsrecht_college zoveel_mogelijk aansluiting_zoekt',
 10: '10_stuks_jongvee_jaar_jongvee jonger',
 11: '11_besluit vastgesteld_verweerder fosfaatrecht_primaire besluit_primaire',
 12: '12_toewijzen_toewijzen verweerder_bedrag college_college verzoek',
 13: '13_omzet_2019_maal ingeschreven_kalenderkwartaal'}

In [12]:
topic_model.visualize_documents(docs, embeddings=sentence_embeddings)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'hovertext': array([' 5. Het standpunt van verzoeksters dat zij iedere rechter wraken die geen machtiging kan overleggen waaruit blijkt dat die rechter een mandaat heeft van de ‘Volkentrust’, kan niet anders worden opgevat dan dat zij bezwaren hebben tegen elke (wrakings)kamer van het College, in welke samenstelling dan ook, zeker in samenhang met de mededeling dat zij de rechtsmacht van het College niet wensen te erkennen (en kennelijk van geen enkele Nederlandse rechterlijke instantie). Een wrakingsverzoek dat neerkomt op een verzoek om wraking van alle rechters van het College, moet buiten behandeling worden gelaten (zie onder meer de uitspraak van het College van 12 april 2012 (ECLI:NL:CBB:2012:BW4809). Bij toewijzing van een dergelijk verzoek zou het verzoek om wraking van de behandelende rechters niet door het College behandeld kunnen worden en zou voorts van afdoening van de aanhangige beroepen geen sprake meer kunnen zijn.',
                                  '1.5 Een politieambtenaar heeft op 3 augustus 2018 een verslag opgemaakt van het binnentreden van de woning van appellant in verband met de hercontrole. Dit verslag vermeldt, voor zover hier van belang, het volgende: “Op donderdag 2 augustus 2018 omstreeks 17:38 uur, ben ik, vergezeld van (…) inspecteur van politie Eenheid Midden-Nederland en (…), hoofdagent van politie Eenheid Midden-Nederland, krachtens een machtiging van de hulpofficier van justitie welke machtiging op 2 augustus 2018 was afgegeven op grond van: artikel2 en 3 van de Algemene wet op het binnentreden. binnengetreden in de woning, (…) bewoond door [naam 1] , zonder toestemming van de bewoner [naam 1] . Bij het binnentreden werd geweld toegepast. Betrokkene weigerde de deur te openen ter controle op dierenwelzijn. Hierbij heeft verbalisant (…) de deur geforceerd zodat wij de woning konden betreden. De woning werd verlaten op donderdag 2 augustus 2018 omstreeks 18:00 uur. (…)”',
                                  '41.20, staan niet vermeld in Bijlage 1 van de Beleidsregel, zodat appellant op basis daarvan niet in aanmerking komt voor een tegemoetkoming. Dat appellant na de afwijzing van zijn aanvraag de SBI-code heeft gecorrigeerd naar',
                                  ...,
                                  '11.55 uur”. Gelet op de in het rapport van bevindingen genoemde controletijden (13.20,',
                                  '2.2 Aan [naam 3] is op 11 september 2007 een vergunning op grond van de Wet milieubeheer verleend voor het houden van 70 melk- en kalfkoeien, 40 stuks vrouwelijk jongvee en 130 stuks vleeskalveren. Uit de vergunning volgt dat al eerder een vergunning is verleend voor het melkvee en de vleesveehouderij.',
                                  None], dtype=object),
              'marker': {'color': '#CFD8DC', 'opacity': 0.5, 'size': 5},
              'mode': 'markers+text',
              'name': 'other',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([5.6858582, 6.516413 , 6.6974163, ..., 9.219135 , 3.7743387, 5.2689815],
                         dtype=float32),
              'y': array([4.300048, 8.304184, 7.886013, ..., 6.653098, 9.277912, 6.133359],
                         dtype=float32)},
             {'hoverinfo': 'text',
              'hovertext': array(['6.3 Anders dan appellante stelt ziet het College niet dat deze uitleg van artikel1, onder kk, van de Msw, zich niet zou verdragen met artikel1 van het EP. Het College verwijst naar de overwegingen in de aangehaalde uitspraken van 25 juni 2019 (ECLI:NL:CBB:2019:244) waarin is ingegaan op de diverse categorieën. Daarbij merkt het College nog op dat geen fosfaatrechten nodig zijn voor stierkalveren op een vleesveebedrijf. Van enige inbreuk op het in artikel1 van het EP bedoelde eigendomsrecht is daarom geen sprake.',
                                  ' 9. De slotsom is dat het hoger beroep ongegrond is. 10. De beslissing